In [8]:
import motmetrics as mm
import numpy as np
import json
import cv2 as cv
from bounding_box_utils.bounding_box_utils import iou

In [9]:

def save_photo(dataset,detector,image_id,gts,det,state):
    img = cv.imread('../%s/%s.jpg'%(dataset,str(image_id).zfill(5)))
    
    i =1
    for s in state:
        cv.putText(img,s, (30,i*70),cv.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 6)
        i+=1
    for gt in gts:
        
    
        cv.rectangle(img, (int(gt[0]), int(gt[1])), (int(gt[2]+gt[0]),int(gt[1]+gt[3])), (255,0,0), 2)
    if (len(det)>0):
        for dd in det:
            
            cv.rectangle(img, (int(dd[0]), int(dd[1])), (int(dd[2]+dd[0] ),int(dd[3]+dd[1])), (0,255,0), 2)
    cv.imwrite('errors/%s/%s.jpg'%(dataset,str(image_id).zfill(5)),img)
def get_summary_dataset_mot(gt_path,dataset,detector,tracker):
    gt={}
    json_ground_trth = "../%s.txt"%gt_path
    gt_file = open(json_ground_trth,"r")
    line = gt_file.readline()
    while(line):
        line = line.replace("\n", "")
        comps = line.split(",")
        image_id = int(comps[0])
        if(image_id not in gt):
            gt[image_id]= []
        gt[image_id].append(np.array([int(comps[1]),float(comps[2]),float(comps[3]),float(comps[4]),float(comps[5])],np.float32))
        line = gt_file.readline()
    gt_file.close()
            
    json_results = "%s/%s/%s.json"%(dataset,detector,tracker)
    dets = {}
    with open(json_results) as json_file:  
        data = json.load(json_file)
        for box in data:
            image_id=box['image_id']
            bbox = box['bbox']
            if(image_id in dets):
                l = list(map(float, bbox))
                l.append(box['id'])
                dets[image_id].append(l)
                
            else:
                dets[image_id]=[]
                l = list(map(float, bbox))
                l.append(box['id'])
                dets[image_id].append(l)
                
    
    acc = mm.MOTAccumulator(auto_id=True)
    
    for key, value in gt.items():
        if(key in dets):
            track_boxes = [v[0:4] for v in dets[key]]
            ids = [d[4] for d in dets[key]]
        else:
            track_boxes = []
            ids=[]
        
        gt_boxes = [vv[1:] for vv in value]
        
        ff = mm.distances.iou_matrix(track_boxes,gt_boxes, max_iou=0.5)
        
            
        frameId =acc.update(
        [t[0] for t in value] ,                 # Ground truth objects in this frame
        ids,                  # Detector hypotheses in this frame

                 ff)
        
    mh = mm.metrics.create()
    summary = mh.compute(acc, metrics=['num_frames', 'num_false_positives','num_misses','num_switches','num_matches','mostly_tracked','partially_tracked','mostly_lost','precision','recall','mota', 'motp','obj_frequencies','pred_frequencies','num_detections','num_objects'], name='acc')
    return summary

In [10]:
def get_summary_dataset(gt_path,dataset,detector,tracker,input_path):
    gt={}
    json_ground_trth = "../%s.json"%gt_path
    with open(json_ground_trth) as json_file:  
        data = json.load(json_file)
        for box in data['annotations']:
            image_id=box['image_id']
            box_id = box['id']
            bbox = box['bbox']
            if(image_id not in gt):
                gt[image_id]= []
            gt[image_id].append([float(bbox[0]),float(bbox[1]),float(bbox[2]),float(bbox[3]),int(box_id)])
            
        
    json_results = "%s/%s/%s.json"%(detector,dataset,tracker)
    dets = {}
    with open(json_results) as json_file:  
        data = json.load(json_file)
        for box in data:
            image_id=box['image_id']
           
            bbox = box['bbox']
            if(image_id in dets):
                
                
                dets[image_id].append([bbox[0],bbox[1],bbox[2],bbox[3],box['id']])
                
            else:
                dets[image_id]=[]
                
               
                dets[image_id].append([bbox[0],bbox[1],bbox[2],bbox[3],box['id']])
    acc = mm.MOTAccumulator(auto_id=True)
    
    for key, value in gt.items():
        
        if(key in dets):
            track_boxes = [v[0:4] for v in dets[key]]
            ids = [d[4] for d in dets[key]]
        else:
            track_boxes = []
            ids=[]
        value_boxes = [g[0:4] for g in value]
        ff = mm.distances.iou_matrix(track_boxes, value_boxes, max_iou=0.5)
        
            
        frameId =acc.update(
        np.arange(len(value_boxes)),                 # Ground truth objects in this frame
        ids,                  # Detector hypotheses in this frame

                 ff)
        
        i=-1
        det_index = -1;
        words = [d for d in acc.mot_events.loc[frameId]['Type']]
        
        #save_photo(input_path,detector,key,value_boxes,track_boxes,words)
            
       
    mh = mm.metrics.create()
    summary = mh.compute(acc, metrics=['num_frames', 'num_false_positives','num_misses','num_switches','num_matches','mostly_tracked','partially_tracked','mostly_lost','precision','recall','mota', 'motp','obj_frequencies','pred_frequencies','num_detections','num_objects'], name='acc')
    return summary

In [11]:
import os
import csv
#ground_truth_file_names = ['data','data_graal1_rev','data_graal2']
#dataset_output_path = ['modd_videos','graal_1_videos','graal_2_videos']
ground_truth_file_names = ['data','data_graal1_rev','data_graal2']
dataset_image_path = ['modd','graal_1','graal_2']
dataset_output_path = ['modd_videos','graal_1_videos','graal_2_videos']
for i in range(len(ground_truth_file_names)):
    
    with open('yolo/%s/mot_results.csv'%(dataset_output_path[i]), 'w', newline='') as csvFile:
        writer = csv.writer(csvFile)
        row = ['Frames','Tot Objects','Tot detections','TP','FN','FP','Track Switches','Precision','Recall','MOTA','MOTP']
        writer.writerow(row)
        for det in ['yolo']:
            for track in ['keypoint_flow','kalman_corners']:
                summary = get_summary_dataset(ground_truth_file_names[i],dataset_output_path[i],det,track,dataset_image_path[i])
                
                row = []
                row.append(summary['num_frames'][0])
                row.append(summary['num_objects'][0])
                row.append(summary['num_detections'][0])
                row.append(summary['num_matches'][0])
                row.append(summary['num_misses'][0])
                row.append(summary['num_false_positives'][0])
                row.append(summary['num_switches'][0])
                row.append(summary['precision'][0])
                row.append(summary['recall'][0])
                row.append(summary['mota'][0])
                row.append(summary['motp'][0])
                writer.writerow(row)



    csvFile.close()

In [9]:
summary = get_summary_dataset_mot('mot_gt','mot_1_videos','def','SORT')


In [10]:
row = ['Frames','Tot Objects','Tot detections','TP','FN','FP','Track Switches','Precision','Recall','MOTA','MOTP']
print(row)
row = []
row.append(summary['num_frames'][0])
row.append(summary['num_objects'][0])
row.append(summary['num_detections'][0])
row.append(summary['num_matches'][0])
row.append(summary['num_misses'][0])
row.append(summary['num_false_positives'][0])
row.append(summary['num_switches'][0])
row.append(summary['precision'][0])
row.append(summary['recall'][0])
row.append(summary['mota'][0])
row.append(summary['motp'][0])
print(row)

['Frames', 'Tot Objects', 'Tot detections', 'TP', 'FN', 'FP', 'Track Switches', 'Precision', 'Recall', 'MOTA', 'MOTP']
[795, 4650, 2066, 1695, 2584, 893, 371, 0.6982088543426833, 0.4443010752688172, 0.1724731182795699, 0.3212888721092702]


In [272]:
cv.putText??